In [1]:
import os
import numpy as np
import torch
torch.set_num_threads(1)
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import argparse
from tqdm import tqdm
import logging
from model import TransformerVAEBaseline
from model import TransformerVAE
from model import TransformerVAEHubert
from utils import AverageMeter
from dataset import get_dataloader
from model.losses import VAELoss, div_loss
from dataset import ReactionDataset


In [2]:
import torch

torch.multiprocessing.set_start_method('spawn')


In [3]:
# model = TransformerVAEBaseline(
#     online=False,
#     img_size=256,    
# #     use_hubert=audio_flag,
#     audio_dim=78, # becomes irrelevant in case for hubert
# #     max_seq_len=751, 
#     seq_len=750)

In [4]:
# checkpoints = torch.load('../data/offline_TransVAE.pth', map_location=torch.device('cpu'))
# state_dict = checkpoints['state_dict']
# model.load_state_dict(state_dict)

In [5]:
model = TransformerVAEHubert(
    online=False,
    img_size=256,    
    use_hubert=True,
    audio_dim=78, # becomes irrelevant in case for hubert
    max_seq_len=751, 
    seq_len=750)

Using HuBERT.


In [6]:
checkpoints = torch.load('./results/hubert_27jun/cur_checkpoint.pth', map_location=torch.device('cpu'))
state_dict = checkpoints['state_dict']
model.load_state_dict(state_dict)

model = model.cuda()

In [7]:
# model = TransformerVAE(
#     img_size=256,    
#     use_hubert=True,
# audio_dim=128, # becomes irrelevant in case for hubert
# max_seq_len=751, 
# seq_len=750)
# checkpoints = torch.load('./results/marlin_hubert_25jun/best_checkpoint.pth', map_location=torch.device('cpu'))
# state_dict = checkpoints['state_dict']
# model.load_state_dict(state_dict)

In [20]:
# import dill as pickle
from dataset import ReactionDataset
from torch.utils.data import DataLoader
from model import TransformerVAE
import pdb
from tqdm import tqdm
# from model import HuBERTEncoder
audio_flag = True

print ('num threads: ', torch.get_num_threads())
# torch.set_num_threads(1)

# model = TransformerVAE(
#     img_size=256,    
#     use_hubert=audio_flag,
# audio_dim=128, # becomes irrelevant in case for hubert
# max_seq_len=751, 
# seq_len=750)

dataset = ReactionDataset('../data', 'data/sample_udiva.csv', clip_length=750, use_raw_audio=audio_flag, load_3dmm_l=True, load_emotion_l=True, mode='val')
shuffle = True 
# sample = dataset[0]
# hubert = HuBERTEncoder(1024)
# hubert = hubert.cuda()

train_loader = DataLoader(dataset=dataset, batch_size=1, shuffle=shuffle, num_workers=2)


# model = model.cuda()
model.eval()
# speaker_video_clip, speaker_audio_clip, _, _, _, _, listener_emotion, listener_3dmm, listener_references = dataset[0]
# print (listener_3dmm.shape)
# print ("enumeratin")
for batch_idx, (speaker_video_clip, 
                speaker_video_orig, 
                speaker_audio_clip, speaker_emotion, _, 
                listener_video_clip, _, listener_emotion, listener_3dmm, listener_references) in enumerate(tqdm(train_loader)):

    print ('speaker video shape: ', speaker_video_clip.shape)
    print ('speaker audio shape: ', speaker_audio_clip.shape)
    print ('listener 3d mm shape: ', listener_3dmm.shape)
    print ("running inference...")
    
    with torch.no_grad():
        listener_3dmm_out, listener_emotion_out, distribution = model(speaker_video_clip.cuda(), speaker_audio_clip.cuda())

print ("done.")
# print ('Batch idx: ', batch_idx)
# train_loader = get_dataloader(args, "data/train.csv", load_ref=False, load_video_l=False)

num threads:  1


  0%|                                                                                                                                                                                | 0/1 [00:00<?, ?it/s]/home/surbhi/anaconda3/envs/marlin/lib/python3.11/site-packages/torch/nn/modules/activation.py:1160: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343970094/work/aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.12s/it]

speaker video shape:  torch.Size([1, 23, 1024])
speaker audio shape:  torch.Size([1, 480000])
listener 3d mm shape:  torch.Size([1, 750, 58])
running inference...


[W CudaIPCTypes.cpp:15] Producer process has been terminated before all shared CUDA tensors released. See Note [Sharing CUDA tensors]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.79s/it]

done.


In [22]:
listener_emotion_out

tensor([[[0.3035, 0.4337, 0.2760,  ..., 0.2348, 0.3445, 0.1702],
         [0.3035, 0.4337, 0.2760,  ..., 0.2348, 0.3445, 0.1702],
         [0.3035, 0.4337, 0.2760,  ..., 0.2348, 0.3445, 0.1702],
         ...,
         [0.3035, 0.4337, 0.2760,  ..., 0.2348, 0.3445, 0.1702],
         [0.3035, 0.4337, 0.2760,  ..., 0.2348, 0.3445, 0.1702],
         [0.3035, 0.4337, 0.2760,  ..., 0.2348, 0.3445, 0.1702]]],
       device='cuda:0')

In [10]:
# model.speaker_behaviour_encoder(speaker_video_clip.cuda(), speaker_audio_clip.cuda())

In [11]:
model.speaker_behaviour_encoder(speaker_video_clip.cuda(), speaker_audio_clip.cuda())

tensor([[[0.0484, 0.0000, 0.0000,  ..., 0.0000, 1.3758, 0.0000],
         [0.1532, 0.0000, 0.0000,  ..., 0.0000, 1.5627, 0.0000],
         [0.2996, 0.0000, 0.0000,  ..., 0.0000, 1.8209, 0.0000],
         ...,
         [0.0000, 0.6367, 0.6167,  ..., 0.4307, 0.0000, 0.6856],
         [0.0000, 0.6367, 0.6167,  ..., 0.4642, 0.0000, 0.6856],
         [0.0000, 0.5422, 0.5230,  ..., 0.4922, 0.0000, 0.5608]]],
       device='cuda:0', grad_fn=<PermuteBackward0>)

In [24]:
vid_enc

tensor([[[ 0.0025, -0.0115, -0.4267,  ..., -0.3654,  0.0414, -0.3364],
         [ 0.0021, -0.0116, -0.4265,  ..., -0.3645,  0.0414, -0.3358],
         [ 0.0029, -0.0121, -0.4264,  ..., -0.3634,  0.0427, -0.3365],
         ...,
         [ 0.0118, -0.0681, -0.3716,  ..., -0.3249,  0.1647, -0.2707],
         [ 0.0114, -0.0670, -0.3699,  ..., -0.3214,  0.1687, -0.2681],
         [ 0.0035, -0.0682, -0.3893,  ..., -0.3670,  0.1548, -0.2583]]],
       grad_fn=<AddBackward0>)

In [14]:
video_encoder = model.speaker_behaviour_encoder.video_encoder

In [15]:
video_encoder

VideoEncoder(
  (Conv3D): ConvBlock(
    (conv1): Conv3d(3, 32, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
    (in1): InstanceNorm3d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (maxpool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 0, 0), dilation=1, ceil_mode=False)
    (conv2): Conv3d(32, 128, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
    (in2): InstanceNorm3d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (relu): ReLU(inplace=True)
    (conv3): Conv3d(128, 128, kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), bias=False)
    (in3): InstanceNorm3d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (conv4): Conv3d(128, 128, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
    (in4): InstanceNorm3d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (conv5): Conv3d(128, 12

In [16]:
model_scripted = torch.jit.script(video_encoder)

In [18]:
model_scripted.save('video_encoder.pt')

In [19]:
model = torch.jit.load('video_encoder.pt')
model.eval()

RecursiveScriptModule(
  original_name=VideoEncoder
  (Conv3D): RecursiveScriptModule(
    original_name=ConvBlock
    (conv1): RecursiveScriptModule(original_name=Conv3d)
    (in1): RecursiveScriptModule(original_name=InstanceNorm3d)
    (maxpool): RecursiveScriptModule(original_name=MaxPool3d)
    (conv2): RecursiveScriptModule(original_name=Conv3d)
    (in2): RecursiveScriptModule(original_name=InstanceNorm3d)
    (relu): RecursiveScriptModule(original_name=ReLU)
    (conv3): RecursiveScriptModule(original_name=Conv3d)
    (in3): RecursiveScriptModule(original_name=InstanceNorm3d)
    (conv4): RecursiveScriptModule(original_name=Conv3d)
    (in4): RecursiveScriptModule(original_name=InstanceNorm3d)
    (conv5): RecursiveScriptModule(original_name=Conv3d)
    (in5): RecursiveScriptModule(original_name=InstanceNorm3d)
  )
  (fc): RecursiveScriptModule(original_name=Linear)
)

In [20]:
model(speaker_video_orig)

tensor([[[ 0.0025, -0.0115, -0.4267,  ..., -0.3654,  0.0414, -0.3364],
         [ 0.0021, -0.0116, -0.4265,  ..., -0.3645,  0.0414, -0.3358],
         [ 0.0029, -0.0121, -0.4264,  ..., -0.3634,  0.0427, -0.3365],
         ...,
         [ 0.0118, -0.0681, -0.3716,  ..., -0.3249,  0.1647, -0.2707],
         [ 0.0114, -0.0670, -0.3699,  ..., -0.3214,  0.1687, -0.2681],
         [ 0.0035, -0.0682, -0.3893,  ..., -0.3670,  0.1548, -0.2583]]],
       grad_fn=<AddBackward0>)